## Writing ome.zarr data from a CZI image file

* Read the CZI image and its metadata into an 6D array
* reduce dimensionality to a 5D array
* write array into an OME-ZARR file

In [1]:
# check if the notebook runs in Google Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ngff-zarr[validate, dask-image]

In [3]:
from czitools.read_tools import read_tools
from czitools.metadata_tools import czi_metadata as czimd
from pathlib import Path
import shutil
import requests
import os
import ngff_zarr as nz
import zarr
print(f"ZARR Verion: {zarr.__version__} NGFF-ZARR Version: {nz.__version__}")

ZARR Verion: 3.1.0 NGFF-ZARR Version: 0.15.0


In [4]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [5]:
if IN_COLAB:
    filepath = os.path.join(os.getcwd(), "data/CellDivision_T3_Z5_CH2_X240_Y170.czi")
    zarr_path = Path(filepath[:-4] + ".ome.zarr")

if not IN_COLAB:
    defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
    filepath = os.path.join(defaultdir, "CellDivision_T3_Z5_CH2_X240_Y170.czi")
    zarr_path = defaultdir / Path(filepath[:-4] + ".ome.zarr")

print(zarr_path)

# check if path exists
remove = True
if zarr_path.exists() and remove:
    shutil.rmtree(zarr_path, ignore_errors=False, onerror=None)

F:\Github\czitools\data\CellDivision_T3_Z5_CH2_X240_Y170.ome.zarr


In [6]:
# get the metadata at once as one big class
mdata = czimd.CziMetadata(filepath)
print("Number of Scenes: ", mdata.image.SizeS)
scene_id = 0

Reading sublocks planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

Number of Scenes:  None


In [7]:
array, mdata = read_tools.read_6darray(filepath)

array = array[scene_id, ...]
print(f"Array Shape: {array.shape}")

Reading sublocks planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

Reading 2D planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

Array Shape: (3, 2, 5, 170, 240)


In [10]:
# create NGFF image from the array
image = nz.to_ngff_image(array.data,
                         dims=["t", "c", "z", "y", "x"],
                         scale={"y": mdata.scale.Y, "x": mdata.scale.X, "z": mdata.scale.Z},
                         name=mdata.filename)



In [11]:
# create multi-scaled, chunked data structure from the image
multiscales = nz.to_multiscales(image, [2, 4], method=nz.Methods.DASK_IMAGE_GAUSSIAN)